In [3]:
import pandas as pd
import numpy as np
import itertools
from scipy import stats

ImportError: dlopen(/Users/williesmalls/anaconda/lib/python3.6/site-packages/scipy/spatial/qhull.cpython-36m-darwin.so, 2): Library not loaded: @rpath/libopenblas.dylib
  Referenced from: /Users/williesmalls/anaconda/lib/python3.6/site-packages/scipy/spatial/qhull.cpython-36m-darwin.so
  Reason: image not found

## Wrangle Data and Create CSV

In [7]:
#read in files
willie_df = pd.read_csv('week3pt2_4pt2.csv')
ken_df = pd.read_csv('DS620_Discussion_Board_Data_Ken.csv')
ken_df.columns = ['Thread_Name','Initial_Post_or_Comment','Author','Response_To','Thread_Content']
willie_df = willie_df[['Thread_Name','Initial_Post_or_Comment','Author','Response_To','Thread_Content']]
jagdish_df = pd.read_csv('DS620_Discussion_Board_Data.csv')
thread_data_df = pd.concat([willie_df, ken_df, jagdish_df])

#Groups 
groups_list = [
    [1, 'Jithendra Seneviratne', 'Sheryl Piechocki'],
    [2, 'Jagdish Chhabria', 'Ken Popkin', 'Willie Smalls'],
    [3, 'Simon Ustoyev', 'Jack Russo', 'Elina Azrilyan', 'Murat Akyildirim'],
    [4, 'Mikhail Kollontai', 'Amber Ferger', 'Zachary Alexander', 'Subhalaxmi Rout'],
    [5, 'Habib Khan', 'Vijaya Cherukuri', 'Priya Shaji', 'Abdellah Ait Elmouden'],
    [6, 'Steven Ellingson'],
    [7, 'Wei Zhou', 'Mia Chen'],
    [8, 'Vanita Thompson', 'Mael Illien', 'Jeremy OBrien']
]

#clean up file
thread_data_df['Author'] = thread_data_df['Author'].replace(['Abdellah Ait Elmouden \n'],'Abdellah Ait Elmouden')
thread_data_df['Response_To'] = thread_data_df['Response_To'].replace(['Abdellah Ait Elmouden \n'],'Abdellah Ait Elmouden')
thread_data_df['Author'] = thread_data_df['Author'].replace(['Elina Azrilyan\xa0'],'Elina Azrilyan')

## review Jeremy O'Brien
thread_data_df['Response_To'] = thread_data_df['Response_To'].replace(["Jeremy O'Brien"], 'Jeremy OBrien')
thread_data_df['Author'] = thread_data_df['Author'].replace(["Jeremy O'Brien"], 'Jeremy OBrien')

## review Jit Seneviratne
thread_data_df['Response_To'] = thread_data_df['Response_To'].replace(['Jit Seneviratne'], 'Jithendra Seneviratne')
thread_data_df['Author'] = thread_data_df['Author'].replace(['Jit Seneviratne'], 'Jithendra Seneviratne')

## review Jagdish Chhabria
thread_data_df['Response_To'] = thread_data_df['Response_To'].replace(['Jagdish Chabria'], 'Jagdish Chhabria')
thread_data_df['Author'] = thread_data_df['Author'].replace(['Jagdish Chabria'], 'Jagdish Chhabria')

#remove spaces
thread_data_df['Response_To'] = thread_data_df['Response_To'].str.strip()
thread_data_df['Author'] = thread_data_df['Author'].str.strip()

#review distinct list of individuals
set(list(thread_data_df['Author']) + list(thread_data_df['Response_To']))

#send dataframe to csv
thread_data_df.to_csv('final_thread_data.csv')

## Remove Initial Posts and Calculate Frequency 

In [13]:
#Read File Back in 
thread_data_df = pd.read_csv('final_thread_data.csv')

#remove all intial posts
thread_data_df = thread_data_df[(thread_data_df['Initial_Post_or_Comment'] != 1)]

#calculate the frequency
thread_data_df = thread_data_df.groupby(['Author','Response_To']).size().reset_index(name='Freq')

#remove where the author is the same as the respond to, Two (2) records, probably missed in review
thread_data_df = thread_data_df.loc[thread_data_df['Author'] != thread_data_df['Response_To']]

thread_data_df

,Author,Response_To,Freq
0,Abdellah Ait Elmouden,Habib Khan,1
1,Abdellah Ait Elmouden,Jack Russo,1
2,Abdellah Ait Elmouden,Ken Popkin,1
3,Abdellah Ait Elmouden,Mael Illien,1
4,Abdellah Ait Elmouden,Mikhail Kollontai,1
...,...,...,...
135,Zachary Alexander,Jack Russo,1
136,Zachary Alexander,Priya Shaji,1
137,Zachary Alexander,Sheryl Piechocki,1
138,Zachary Alexander,Steven Ellingson,1


In [9]:
#create list of poster pairs and values
thread_data_list = thread_data_df.values.tolist()
thread_data_list

#Create a list of distinct class members
member_list = list(set(thread_data_df['Author'].values.tolist() + thread_data_df['Response_To'].values.tolist()))
member_combo_list = [list(combo) for combo in itertools.combinations(member_list, 2)]

#Create new list with row for count. 
member_combo_list2 = []
for pair in member_combo_list:
    pair.append(0)
    pair.append(0)
    member_combo_list2.append(pair)

    
#loop through pair list
for thread_pair_list in thread_data_list:

    for pair_id, pairs in enumerate(member_combo_list2):
        person_one = pairs[0]
        person_two = pairs[1]
        freq = pairs[2]
        
        #sum number of interactions
        if all(x in thread_pair_list for x in [person_one, person_two]):
            member_combo_list2[pair_id][2] = freq + thread_pair_list[2]
            

conversation_list = []
#remove all non conversation people
for i in member_combo_list2: 
    if i[2]>0:
        conversation_list.append(i)

conversation_list
# groups_list

for idx, pair in enumerate(conversation_list):
    person_one = pair[0]
    person_two = pair[1]
    
    for group in groups_list:
        if all(x in group for x in [person_one, person_two]):
            conversation_list[idx][3] = 1

conversation_df = pd.DataFrame(conversation_list)
conversation_df.columns = ['person_one', 'person_two', 'freq', 'group']
conversation_df

,person_one,person_two,freq,group
0,Subhalaxmi Rout,Priya Shaji,2,0
1,Subhalaxmi Rout,Simon Ustoyev,2,0
2,Subhalaxmi Rout,Murat Akyildirim,1,0
3,Subhalaxmi Rout,Mia Chen,1,0
4,Subhalaxmi Rout,Mikhail Kollontai,1,1
...,...,...,...,...
101,Amber Ferger,Elina Azrilyan,1,0
102,Amber Ferger,Vanita Thompson,2,0
103,Jeremy OBrien,Abdellah Ait Elmouden,1,0
104,Jeremy OBrien,Vijaya Cherukuri,1,0


In [3]:
groups_df = conversation_df[(conversation_df.group==1)]['freq']
nongroups_df = conversation_df[(conversation_df.group==0)]['freq']
scipy.stats.ttest_ind(groups_df, nongroups_df)

NameError: name 'conversation_df' is not defined